### Helper Utils
1,2,3


In [68]:
import numpy as np
def suffixArrayConstruction(text):
    ordered_pairs = sorted([(text[i:],i) for i in range(len(text))])
    suffix_array = [pair[1] for pair in ordered_pairs]
    return suffix_array, ordered_pairs

def greaterThan(p, p_):
    if (p_[:len(p)] < p):
        return True
    return False

def equalPatterns(p,p_):
    if p_[:len(p)] == p :
        return True
    return False

def ApproximatePatternMatching(Pattern, Text, d):
    positions = []
    patterns_list = []
    for i in range(len(Text)-len(Pattern)+1):
        Pattern_ = Text[i:i+len(Pattern)]
        if HammingDistance(Pattern_, Pattern) <= d:
            positions.append(i)
            patterns_list.append(Pattern_)

    return patterns_list
def HammingDistance(p, q):
    count = 0
    for (i,j) in zip(p,q):
        if i != j:
            count += 1
    return count

### Pattern Matching with the Suffix Array

In [13]:
def patternMatchingSA(text, patterns,suffixArr):
    minidx = 0
    maxidx = len(text)-1
    while minidx <= maxidx:
        mididx = (minidx+maxidx)//2
        if greaterThan(pattern,text[suffixArr[mididx]:]):
            minidx = mididx+1
        else:
            maxidx = mididx-1

    if equalPatterns(pattern,text[suffixArr[mididx]:suffixArr[mididx]+len(pattern)]):
        first = suffixArr[minidx]
    else:
        return 

    maxidx = len(text)-1
    while minidx <= maxidx:
        mididx = (minidx+maxidx)//2
        if text[suffixArr[mididx]:].startswith(pattern):
            minidx = mididx+1
        else:
            maxidx = mididx-1
    last= suffixArr[maxidx]
    return (first, last)


In [12]:
f = open('test.txt')
s = f.readline().strip()
patterns = [i.strip() for i in f.readlines()]
sArray,_ = suffixArrayConstruction(s)
positions = []
for pattern in patterns:
    pair = patternMatchingSA(s, pattern, sArray)
    if pair is not None:
        positions.append(pair)
print(' '.join([str(pos) for pos in sorted([x for t in positions for x in t])]))


1 4 11 15


### Construct the Burrows-Wheeler Transform of a String

In [24]:
def constructBWTstring(text):
    cycles = []

    for i in range(len(text)):
        shift = text[i:] + text[:i]
        cycles.append(shift)

    cycles = sorted(cycles)
    bwt = ''
    for cycle in cycles:
        bwt += cycle[-1]
    return bwt 
constructBWTstring("GCGTGCCTGGTCA$")

'ACTGGCT$TGCGGC'

### Reconstruct a String from its Burrows-Wheeler Transform

In [5]:
def index_symbol (column):
    idx_tuple = {symbol:0 for symbol in column}
    col = []
    for symbol in column:
        idx_tuple[symbol]+=1
        col.append((symbol, idx_tuple[symbol]))
    print(col)
    return col
def decodeFromBWT(bwt):
    first_col = ''.join(sorted([i for i in bwt]))
    first_index_tuple = index_symbol(first_col)
    last_col = bwt
    last_index_tuple = index_symbol(last_col)
    recons =''
    symbol = ('$',1)

    while len(recons)<len(bwt):
        symbol = first_index_tuple[last_index_tuple.index(symbol)]
        # print("symbol -> ", symbol)
        recons += symbol[0]
    # return recons[:-1]
    

In [6]:
decodeFromBWT("smnpbnnaaaaa$a")

[('$', 1), ('a', 1), ('a', 2), ('a', 3), ('a', 4), ('a', 5), ('a', 6), ('b', 1), ('m', 1), ('n', 1), ('n', 2), ('n', 3), ('p', 1), ('s', 1)]
[('s', 1), ('m', 1), ('n', 1), ('p', 1), ('b', 1), ('n', 2), ('n', 3), ('a', 1), ('a', 2), ('a', 3), ('a', 4), ('a', 5), ('$', 1), ('a', 6)]


### Generate the Last-to-First Mapping of a String

In [4]:
def lastTofirstMap(transform, idx):
    last_col = transform
    first_col = sorted(transform)
    last_index_tuple = index_symbol(last_col)
    first_index_tuple = index_symbol(first_col)

    mapped_idx = first_index_tuple.index(last_index_tuple[idx])
    return mapped_idx

lastTofirstMap("T$GACCA", 3)  

1

### Find All Occurrences of a Collection of Patterns in a String

In [65]:
def partialSuffixArray(text,k=5) :
    _, sufArrayPairs = suffixArrayConstruction (text) 
    pSAidx = []
    for i in range(len(sufArrayPairs)):
        if sufArrayPairs[i][1]%k == 0:
            pSAidx.append((i, sufArrayPairs[i][1]))
    return pSAidx

def first_occurence(last_column):
    return dict(zip(*np.unique(sorted(last_column), return_index=True)))

def countSymbol(symbol,idx, column):
    return column[:idx].count(symbol)

def checkpointArray(column, k=5):
    frequencyDict = {i:[] for i in "ACGT"}
    for i in range(k, len(column), k):
        for base in "ACGT":
            frequencyDict[base].append(countSymbol(base,i,column))
    return frequencyDict

def betterBWTmatching(last_column, pattern, cp_array):
    top = 0
    bottom = len(last_column) - 1
    firsts = first_occurence(last_column)

    while top <= bottom:
        if pattern :
            symbol = pattern[-1]
            pattern = pattern[:-1]
            if symbol in last_column[top:bottom+1]:
                top = firsts[symbol] + countSymbol(symbol, top, last_column)
                bottom = firsts[symbol] + countSymbol(symbol, bottom+1, last_column) - 1
            else:
                return 0
        else:
            return top, bottom

def multiplePatternMatching(text, patterns) :
    sufArray,_ = suffixArrayConstruction(text)
    print("SA -> ", sufArray)
    bwt = constructBWTstring(text)
    print("BWT -> ", bwt)
    # last_column = list(bwt)
    indices = []
    cp_array = checkpointArray(bwt)
    print("cpArray -> ", cp_array)
    for pattern in patterns:
        top, bottom =  betterBWTmatching(bwt, pattern, cp_array)
        print(top,"--------", bottom)
        for i in range(top, bottom+1):
            indices.append(sufArray[i])

    return sorted(indices)



In [66]:
f = open("test.txt","r")
s = f.readline().strip()
patterns = f.read().strip().split()

multiplePatternMatching(s, patterns)

SA ->  [10, 0, 11, 1, 9, 13, 3, 14, 15, 4, 16, 5, 17, 6, 18, 8, 12, 2, 7]
BWT ->  CTAATTTCGCGGGGGTAAG
cpArray ->  {'A': [2, 2, 2], 'C': [1, 3, 3], 'G': [0, 1, 6], 'T': [2, 4, 4]}
2 -------- 3
8 -------- 9


[1, 4, 11, 15]

### Multiple Approximate Pattern Matching Problem

In [72]:
def extend_kmer(index, top, bottom, sufArray):
    return [int(sufArray[i]) - index for i in range (top, bottom + 1)]

def multipleApproxPatternMatching(text, patterns, d):
    sufArray,_ = suffixArrayConstruction(text)
    print("SA -> ", sufArray)
    bwt = constructBWTstring(text)
    print("BWT -> ", bwt)
    last_column = list(bwt)
    indices = []
    cp_array = checkpointArray(bwt)
    print("cpArray -> ", cp_array)
    for pattern in patterns:
        new_pattern = ApproximatePatternMatching(pattern, text, d)
        for p in new_pattern:
            top, bottom =  betterBWTmatching(bwt, p, cp_array)
            print(top,"--------", bottom)
            for i in range(top, bottom+1):
                indices.append(sufArray[i])
    return sorted(indices)

In [73]:
f = open("test.txt","r")
text = f.readline().rstrip() + '$'
patterns = [line.rstrip() for line in f.readline().split()]
d = int(f.readline())

multipleApproxPatternMatching(text, patterns, d)

SA ->  [12, 0, 7, 2, 1, 5, 8, 4, 11, 6, 3, 10, 9]
BWT ->  T$TCAGATTCATC
cpArray ->  {'A': [1, 2], 'C': [1, 2], 'G': [0, 1], 'T': [2, 4]}
3 -------- 3
2 -------- 2
6 -------- 6
12 -------- 12
7 -------- 7
7 -------- 7
9 -------- 9


[2, 4, 4, 6, 7, 8, 9]